<a href="https://colab.research.google.com/github/jindojojin/Analyst/blob/master/QuangLinh_Interpreter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Program input

begin
  int a ;
  int b ;
  int A123a ;
  int aB123 ;
  a = 200 ;
  b = 200 ;
  A123a = a * b ;
  while ( a == b ) do 
   begin
      print a ;
   end ;
end

# Scanner

In [ ]:
#@title Get program input & declare common classes
from google.colab import _message
# Load the notebook JSON.
nb = _message.blocking_request('get_ipynb')
# Get source code from 'Program input' Session.
SOURCE_CODE_INPUT = ''.join(nb['ipynb']['cells'][1]['source'])

class WordType: # Enum
  Operator = 'Op'
  ID = 'Id'
  Number = 'Num'
  Keyword = 'Keyword'
  Type = 'Type'
  Assign = '='

class Word:
  def __init__(self, name, wordType):
    self.name = name
    self.wordType = wordType

### Automat

In [62]:
# @title Build NFA
class NFA:
  def __init__(self):
    self.StartState = 0 # S0
    self.AcceptingStates = []  # S1 = ending = accepting state
    self.TransMap = {} 
    self.stateCount = 1 # init with start = 0 & end = 1

def _transition2TransMapkey(state, char):
  return f"{200*state + ord(char)}" # 200 > num of ascii code

def _TransMapkey2transition(key):
  index = int(key)
  charCode = chr(index % 200)
  state = index // 200
  return [
      state,
      charCode
  ]

def _shiffStateNFA(nfa,newStart=0): # Bien NFA co startState = oldStart thanh NFA co startState = newStart
  offset = newStart - nfa.StartState
  newNFA = NFA()
  newNFA.StartState = newStart
  newNFA.stateCount = nfa.stateCount
  for state in nfa.AcceptingStates:
    newNFA.AcceptingStates.append(state + newStart)
  for key in nfa.TransMap:
    transition = _TransMapkey2transition(key)
    newNFA.TransMap[_transition2TransMapkey(transition[0]+ offset, transition[1])] = nfa.TransMap[key] + offset
  return newNFA;

def _andNFA(list_nfa = []):  # ab
  resultNFA = NFA()
  # for nfa in list_nfa:
  return resultNFA

def _orNFA(list_nfa = []):   # a|b
  result = NFA()
  return result

def _kleene_closure_NFA(nfa):# a*
  result = NFA()
  return result

def createNFA(word):
  nfa = NFA()
  for c in word:
    nfa.stateCount += 1
    nfa.TransMap[_transition2TransMapkey(nfa.stateCount-1,c)] = nfa.stateCount # move to next character (new state)
  nfa.AcceptingStates = [nfa.stateCount]
  return nfa

nfas = []
for keyword in ['begin','end','print', 'while', 'do' , ';']:
  nfa = createNFA(keyword)
  print ("shiff------",_shiffStateNFA(nfa,5).TransMap)
  print ("old--------",nfa.TransMap)
  nfas.append(createNFA(keyword))

# create nfa for ID

# create nfa for Number

finalNFA = _orNFA(nfas)

shiff------ {'1298': 7, '1501': 8, '1703': 9, '1905': 10, '2110': 11}
old-------- {'298': 2, '501': 3, '703': 4, '905': 5, '1110': 6}
shiff------ {'1301': 7, '1510': 8, '1700': 9}
old-------- {'301': 2, '510': 3, '700': 4}
shiff------ {'1312': 7, '1514': 8, '1705': 9, '1910': 10, '2116': 11}
old-------- {'312': 2, '514': 3, '705': 4, '910': 5, '1116': 6}
shiff------ {'1319': 7, '1504': 8, '1705': 9, '1908': 10, '2101': 11}
old-------- {'319': 2, '504': 3, '705': 4, '908': 5, '1101': 6}
shiff------ {'1300': 7, '1511': 8}
old-------- {'300': 2, '511': 3}
shiff------ {'1259': 7}
old-------- {'259': 2}


In [ ]:
# @title Convert NFA to DFA

In [ ]:
#@title Automat class
class Automat:
  
  def __init__(self):
    self.reset()

  def reset(self):
    self.currentState = 0
    self.currentWord = ''

  def transition(self, c):
    stop = False
    error = False
    if(c == '' or c == '\n' or c == '\t'):
      stop = True
      if(not nfa.AcceptingStates.includes(self.currentState)):
        error = True
    else:
      self.currentWord += c
      transKey = f"{self.currentState}-{c}"
      if(transKey in nfa.TransMap):
        self.currentState = nfa.TransMap[transKey]
      else:
        stop = True
        error = True
  
    result = {
        'stop': stop,
        'error': error,
        'word': self.currentWord
    }

    if(stop):
      self.reset()

    return result



[6, 5, 8]
{'1-b': 2, '2-e': 3, '3-g': 4, '4-i': 5, '5-n': 6, '3-n': 4, '4-d': 5, '3-p': 4, '4-r': 5, '5-i': 6, '6-n': 7, '7-t': 8}


### Scanner implementation

In [ ]:
class Scanner:
  def __init__(self):
    self.words = []
  
  def scan(self):
    print(">>>>>>>>>>> start scan procedure")
    automat = Automat()
    endingPos = len(SOURCE_CODE_INPUT) -1
    for i in range(endingPos):
      c = SOURCE_CODE_INPUT[i]
      result = automat.transition(c)
      if ( result['stop']):
        if (result['error']):
          #Todo tao ham thong bao loi
          print("Chuong trinh khong hop le")
          break
        else:
          word = result['word']
          self.words.append(Word(word, WordType.Operator))
    print(">>>>>>>>>>> finish scan procedure")

  def printScanResult(self):
    for word in self.words:
      print(f"{word.name} : {word.wordType}")



# Main

In [ ]:
scanner = Scanner()
scanner.scan()

>>>>>>>>>>> start scan procedure
Da co loi xay ra
>>>>>>>>>>> finish scan procedure
